----> This is a notebook for Final Recommendation System.<br>
----> STEP 1:<br>
Importing all the necessary modules.

In [2]:
import numpy as np
import pandas as pd
import warnings
import pickle as pkl
import string
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

Step 2: <br>
Loading all processed data and models

In [3]:
with open("/kaggle/input/data-mining-processed-data/yelp_recommendation_model_Q.pkl", "rb") as f:
    Q = pkl.load(f)
    
with open("/kaggle/input/data-mining-processed-data/yelp_recommendation_model_userid_vectorizer.pkl", "rb") as f:
    userid_vectorizer = pkl.load(f)

restaurant = pd.read_csv("/kaggle/input/data-mining-processed-data/restaurant.csv", index_col=0)
restaurant_df= pd.read_csv("/kaggle/input/data-mining-processed-data/restaurant_cluster_df.csv", index_col=0)
cosine_similarity_df = pd.read_csv("/kaggle/input/data-mining-processed-data/cosine_similarity_restaurant_df.csv", index_col=0)

Code for text processing

In [4]:
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])    

Function for restaruant name as input and output is recommended restaurants

In [5]:
def recommender(name):
    try: 
        given_restaurant=restaurant_df[restaurant_df["name"]==name]
        same_cluster = restaurant_df[restaurant_df["cluster"] == list(given_restaurant.cluster.items())[0][1]]
        distance = np.sqrt((same_cluster.latitude - list(given_restaurant.cluster.items())[0][1])**2 + (same_cluster.longitude - list(given_restaurant.cluster.items())[0][1])**2)
        same_cluster["distance"] = distance
        same_cluster.sort_values('distance',inplace=True)
        recommended_restaurant = same_cluster.iloc[1:6]
        for i in recommended_restaurant["business_id"]:
            print(restaurant[restaurant['business_id']==i]['name'].iloc[0])
            print(restaurant[restaurant['business_id']==i]['categories'].iloc[0])
            print(str(restaurant[restaurant['business_id']==i]['stars'].iloc[0])+ ' '+str(restaurant[restaurant['business_id']==i]['review_count'].iloc[0]))
            print('')
        try:
            for i in cosine_similarity_df.iloc[cosine_similarity_df[name].sort_values(ascending=False).index].name[1:6].items():
                print(restaurant[restaurant['name']==i[1]]['name'].iloc[0])
                print(restaurant[restaurant['name']==i[1]]['categories'].iloc[0])
                print(str(restaurant[restaurant['name']==i[1]]['stars'].iloc[0])+ ' '+str(restaurant[restaurant['name']==i[1]]['review_count'].iloc[0]))
                print('')
        except:
            pass
    except:
        print("Restaurant Doesn't Exists!!\n")     

In [6]:
recommender("Chick-fil-A")

On Rye Deli
Delis, Restaurants
3.5 37

Cedar Cafe
Sandwiches, Fast Food, Salad, Greek, Restaurants, Mediterranean
4.5 87

99 Restaurants
American (New), Desserts, Food, Restaurants, Bars, Nightlife, Pubs, American (Traditional)
3.0 31

Chipotle Mexican Grill
Restaurants, Mexican, Fast Food
2.5 84

Stars On Hingham Harbor
Restaurants, Nightlife, Seafood, American (Traditional), Bars
4.0 260

Tin Drum Asian Kitchen - The Village at Druid Hills
Food, Chinese, Thai, Asian Fusion, Restaurants
3.0 131

Panera Bread
Restaurants, Sandwiches, Breakfast & Brunch, Bagels, Soup, Bakeries, Food, Salad
2.5 55

Nature's Table Bistro
Restaurants, Sandwiches, Fast Food, American (Traditional), Salad
3.5 150

Hong Kong City
Restaurants, Chinese
3.5 16

Mellow Mushroom Atlanta - Buckhead
Nightlife, Sandwiches, Pizza, Bars, Restaurants
3.5 140



In [12]:
recommender("Hunan Lion")

Asaka Japanese & Asian Cuisine
Restaurants, Japanese, Asian Fusion
4.5 52

Feng Shui
Restaurants, Buffets, Seafood, Chinese, Sushi Bars, Japanese
2.5 85

The Four's Norwell
American (Traditional), Bars, Sports Bars, Restaurants, Nightlife
3.0 142

Wild Ginger
Thai, Food Delivery Services, Food, Restaurants
4.5 136

Not Your Average Joe's
Restaurants, American (New)
3.5 195

Pita Hut Grille
Mediterranean, Middle Eastern, Restaurants
4.0 162

Hounddog's Three Degree Pizza
Pizza, Restaurants
4.0 518

Tucci's
Venues & Event Spaces, Steakhouses, American (New), Seafood, Restaurants, American (Traditional), Event Planning & Services, Italian
4.0 252

Blue Ginger Asian Fusion Bistro
Noodles, Sushi Bars, Restaurants, Asian Fusion
3.5 170

Cameron's American Bistro
Bars, Restaurants, Nightlife, Gluten-Free, American (New)
4.0 157



Function for user desirable text as input and output is recommended restaurants.

In [7]:
def text_Recommender(text):
    test_df= pd.DataFrame([text], columns=['text'])
    test_df['text'] = test_df['text'].apply(text_process)
    test_vectors = userid_vectorizer.transform(test_df['text'])
    test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

    predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
    topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:5]

    for i in topRecommendations.index:
        print(restaurant[restaurant['business_id']==i]['name'].iloc[0])
        print(restaurant[restaurant['business_id']==i]['categories'].iloc[0])
        print(str(restaurant[restaurant['business_id']==i]['stars'].iloc[0])+ ' '+str(restaurant[restaurant['business_id']==i]['review_count'].iloc[0]))
        print('')        

In [9]:
text_Recommender("I am craving for Pizza")

Santarpio's Pizza
Restaurants, Barbeque, Pizza
4.0 1281

Hounddog's Three Degree Pizza
Pizza, Restaurants
4.0 518

More Home Slice Pizza
Restaurants, Pizza
4.5 512

Posto
Restaurants, Seafood, Italian, Pizza
4.0 910

Jack's Pizza & Wings
Music Venues, Pizza, Arts & Entertainment, Chicken Wings, Nightlife, Pubs, Sandwiches, Bars, Restaurants
3.5 626



In [10]:
text_Recommender("I want to eat italian food with beautiful views")

Carlo's Cucina Italiana
Salad, Italian, Desserts, Food, Restaurants
4.5 693

The Mad Italian
Restaurants, Italian, Sandwiches, American (Traditional)
4.0 579

Maggiano's Little Italy
Italian, Caterers, Venues & Event Spaces, Restaurants, Event Planning & Services, Nightlife, Bars
3.5 908

Baraonda Italian Restaurant
Italian, Pizza, Restaurants
3.5 542

The Florentine Cafe
Cafes, Restaurants, Italian
3.5 497



In [11]:
text_Recommender("A peaceful place with authentic lunch")

Border Grill Fresh-Mex
Latin American, Mexican, Restaurants, Tex-Mex
4.5 1203

El Tenampa Mexican Restaurant
Restaurants, Mexican
4.0 582

Thai Cuisine
Restaurants, Thai
4.0 751

Hollerbach's Willow Tree Cafe
Restaurants, Sandwiches, Nightlife, Breakfast & Brunch, German, Beer, Wine & Spirits, Food, Beer Gardens, Desserts, Bars
4.5 991

Tasty Wok
Restaurants, Soup, Noodles, Chinese, Cantonese, Barbeque
4.0 490

